In [ ]:
from keras.models import Sequential, Model
from keras.layers import Conv3D, BatchNormalization, Flatten, Dropout, MaxPooling3D, LSTM, Dense, ZeroPadding3D
from keras.regularizers import L2

In [ ]:
# load data
import os
import pandas as pd
import cv2
import numpy as np

num_frames = 20 # changing requires model refactoring

def load_video(directory):
    cap = cv2.VideoCapture(directory)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            frames.append(cv2.cvtColor(frame,cv2.COLOR_BGR2RGB))
        else:
            break
    cap.release()
    return np.array(frames)/255

def load_txt(directory):
    df = pd.read_csv(directory)
    dataset = df.iloc[1:,1:]
    return np.array(dataset)

def load_data(directory):
    label_count = 0
    X_fr_train = []
    X_mk_train = []
    y_train = []

    X_fr_test =  []
    X_mk_test = []
    y_test = []

    list_label = os.listdir(directory)
    for label in list_label:
        count = 0
        subpath = os.path.join(directory,label)
        files = os.listdir(subpath)

        files_txt = [txt for txt in files if txt.endswith(".txt")]
        files_mp4 = [mp4 for mp4 in files if mp4.endswith(".mp4")]

        if len(files_txt)!=len(files_mp4):
            raise RuntimeError("The amount of .txt and .mp4 files are not equal. Found {} .mp4 but {} .txt".format(len(files_mp4),len(files_txt)))
        
        n = len(files_txt)
        
        for i in range(n):
            frames = load_video(os.path.join(subpath,files_mp4[i]))
            marks = load_txt(os.path.join(subpath,files_txt[i]))
            n_samples = marks.shape[0]
            if count>=1000:
                break
            for j in range(num_frames, n_samples, 5):
                count+=1
                if count%4!=0:
                    X_fr_train.append(frames[j-num_frames:j,::])
                    X_mk_train.append(marks[j-num_frames:j,:])
                    y_train.append(label_count)    
                    continue
                X_fr_test.append(frames[j-num_frames:j,::])
                X_mk_test.append(marks[j-num_frames:j,:])
                y_test.append(label_count)
        label_count+=1
    return X_fr_train,X_mk_train, y_train, X_fr_test, X_mk_test, y_test, list_label

In [ ]:
def create_LSTM():
    return Sequential([
        LSTM(126,return_sequences=True,kernel_regularizer=L2()),
        BatchNormalization(),
        Dropout(.2),
        
        LSTM(63,return_sequences=True,kernel_regularizer=L2()),
        BatchNormalization(),
        Dropout(.2),

        Flatten()
    ])

def create_Conv3D(): # designed for 20 128 128 3
    return Sequential([
        Conv3D(16,
                kernel_size=(3,3,3),
                strides=(1,1,1),
                padding="same",
                data_format="channels_last",
                activation="relu",
                kernel_regularizer=L2()),
        MaxPooling3D(pool_size=(2,2,2),
                strides=(1,2,2)),
        BatchNormalization(),

        Conv3D(32,
                kernel_size=(3,3,3),
                strides=(1,1,1),
                padding="same",
                data_format="channels_last",
                activation="relu",
                kernel_regularizer=L2()),
        MaxPooling3D(pool_size=(2,2,2),
                strides=(1,2,2)),
        BatchNormalization(),

        Conv3D(64,
                kernel_size=(3,3,3),
                strides=(1,1,1),
                padding="same",
                data_format="channels_last",
                activation="relu",
                kernel_regularizer=L2()),
        MaxPooling3D(pool_size=(2,2,2),
                strides=(2,2,2)),
        BatchNormalization(),

        Conv3D(128,
                kernel_size=(3,3,3),
                strides=(1,1,1),
                padding="same",
                data_format="channels_last",
                activation="relu",
                kernel_regularizer=L2()),
        MaxPooling3D(pool_size=(2,2,2),
                strides=(2,2,2)),
        BatchNormalization(),

        ZeroPadding3D(((1,2),(0,0),(0,0)), data_format="channels_last"),
        MaxPooling3D(pool_size=(2,2,2),
                strides=(2,2,2)),
        Flatten()
    ])

In [ ]:
import tensorflow as tf

# first build model

class ConvLSTM(Model):
    def __init__(self, out_classes):
        super().__init__()
        self.branch1 = create_Conv3D()
        self.branch2 = create_LSTM()

        self.classifier = Sequential([
            Dense(out_classes,activation="softmax")
        ])

    def call(self, inp):
        inp_frame = self.branch1(inp[0])
        inp_mark = self.branch2(inp[1])
        out = tf.concat((inp_frame,inp_mark),axis=1)
        out = self.classifier(out)
        return out

In [ ]:
directory = "Data_processed/"
X_fr_train, X_mk_train, y_train, X_fr_test, X_mk_test, y_test, list_label = load_data(directory)

print(len(y_train),len(y_test))
y_train, y_test = np.array(y_train) , np.array(y_test)
X_fr_train = np.array(X_fr_train)
X_fr_test = np.array(X_fr_test)
X_mk_train = np.array(X_mk_train)
X_mk_test = np.array(X_mk_test)

In [ ]:
import numpy as np
# conv3d = create_Conv3D()
input1 = np.random.randn(16,20,128,128,3)
input2 = np.random.randn(16,20,126)
model = ConvLSTM(28)
output = model([input1,input2])
print(output.shape)
model.load_weights("testconv3dlstm_PBL5.weights.h5")

In [ ]:
input1 = np.random.randn(1,20,128,128,3)
input2 = np.random.randn(1,20,126)
output = model([input1,input2])

In [ ]:
import tensorflow as tf
from keras.optimizers import SGD, Adamax
from keras.callbacks import ModelCheckpoint
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=Adamax(),
              metrics=["accuracy"])
checkpoint = ModelCheckpoint(

    save_best_only=True,
    save_weights_only=False,filepath="testconv3dlstm_PBL5_functional_apis.keras",
    # save_weights_only=True,filepath="testconv3dlstm_PBL5_functional_apis.weights.h5",
)

In [ ]:
history = model.fit([X_fr_train,X_mk_train],y_train,
                    epochs=20,
                    batch_size=16,
                    validation_data=([X_fr_test,X_mk_test],y_test),
                    callbacks = [checkpoint])

In [ ]:
model.load_weights("testconv3dlstm_PBL5.weights.h5")
model.evaluate([X_fr_train,X_mk_train],y_train)